In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

output_dir = '/home/maria/AllenData'
manifest_path = os.path.join(output_dir, "manifest.json")

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)
session=cache.get_session_data(831882777)
lfp = session.get_lfp(832810578)
session.spike_times

In [1]:
from transformers import AutoImageProcessor, ViTModel

import torch

from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image", trust_remote_code=True)

image = dataset["test"]["image"][0]

image_processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224')

model = ViTModel.from_pretrained('google/vit-base-patch16-224')

inputs = image_processor(image, return_tensors="pt")

with torch.no_grad():

    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

list(last_hidden_states.shape)

/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maria/NeuroGarage/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[1, 197, 768]

In [2]:
# Calculate the size in bytes
model_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())

# Convert to megabytes
model_size_mb = model_size_bytes / (1024 ** 2)
print(f"Model size: {model_size_mb:.2f} MB")

Model size: 329.55 MB


In [4]:
from torchinfo import summary

# Move the model to CPU (or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Print the summary
summary(model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                             Output Shape              Param #
ViTModel                                           [1, 768]                  --
├─ViTEmbeddings: 1-1                               [1, 197, 768]             152,064
│    └─ViTPatchEmbeddings: 2-1                     [1, 196, 768]             --
│    │    └─Conv2d: 3-1                            [1, 768, 14, 14]          590,592
│    └─Dropout: 2-2                                [1, 197, 768]             --
├─ViTEncoder: 1-2                                  [1, 197, 768]             --
│    └─ModuleList: 2-3                             --                        --
│    │    └─ViTLayer: 3-2                          [1, 197, 768]             7,087,872
│    │    └─ViTLayer: 3-3                          [1, 197, 768]             7,087,872
│    │    └─ViTLayer: 3-4                          [1, 197, 768]             7,087,872
│    │    └─ViTLayer: 3-5                          [1, 197, 768]             7,087,8